In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
from PedAPI import PED

In [3]:
import psycopg2

/home/olksndr/enviroments/tfdeeplearning/lib/python3.5/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
conn = psycopg2.connect("host = localhost dbname = postgres user = postgres")

curr = conn.cursor()

In [5]:
curr.execute("""
select distinct(STORE_NUM) from transactions;

""")

In [6]:
unique_stores = curr.fetchall()

In [7]:
curr.execute("""
select distinct(UPC) from transactions;

""")

In [8]:
unique_products = curr.fetchall()

In [9]:
conn.close()

In [10]:
conn = psycopg2.connect("host = localhost dbname = postgres user = postgres")

curr = conn.cursor()

In [11]:
with open('df_columns.txt','r') as file:
    cols = file.read()
columns = cols.split(',')[:12]

In [12]:
result = pd.DataFrame()

for unique_store in unique_stores:
    for unique_product in unique_products:
        
        
        curr.execute("""
            select * from transactions
            where UPC = %s AND STORE_NUM = %s
            """, (unique_product[0], unique_store[0]))
        
        sample = curr.fetchall()
        
        sample = pd.DataFrame(sample, columns=columns)
        
        if sample.shape[0] < 99:
            continue
        elif sample.isnull().values.any():
            sample = sample.dropna()
        
        ped_est = PED(sample)
        ped_est.perform_fit()
        ped_est.get_estimates()
        
        result = pd.concat((result, pd.DataFrame(ped_est.get_results())))
        #result.to_csv('result.csv')
result.reset_index().drop('index', axis = 1).to_csv()
conn.close()

In [13]:
result.describe()

,STORE_NUM,UPC,current_est_revenue,current_price,estimated_revenue,mean_revenue_increase,recommended_price,week_1_price,week_2_price
count,3391.000000,3.391000e+03,3391.000000,3391.000000,3391.000000,3391.000000,3391.000000,3391.000000,3391.000000
mean,14696.089354,1.119345e+10,58.092524,3.532265,88.128337,2.590241,4.012159,3.523288,3.519387
std,8907.934749,2.201341e+10,56.116882,1.507326,111.803868,5.521668,1.836055,1.462578,1.461249
min,367.000000,1.111009e+09,2.336526,1.170000,4.438793,-0.779957,0.860000,1.190000,1.160000
25%,6187.000000,1.111087e+09,18.487161,2.505000,25.369175,0.285198,2.600000,2.545000,2.560000
50%,15541.000000,3.700032e+09,41.389553,3.290000,52.613622,0.925626,3.680000,3.290000,3.290000
75%,23067.000000,7.218063e+09,76.503429,4.590000,108.322678,2.279370,4.790000,4.590000,4.535000
max,29159.000000,8.849121e+10,523.301411,11.460000,1513.949739,66.511921,8.890000,11.460000,11.460000
